In [1]:
import cv2,os
import numpy as np
import pickle
from tensorflow import keras

from keras.models import load_model

Using TensorFlow backend.


In [ ]:
source      = cv2.VideoCapture(0)
face_clsfr  = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model       = load_model('my_model')

#model = pickle.load(open('finalized_model.sav', 'rb'))
#initial_model = load_model('my_model1')

labels_dict = {0:'mask',1:'no_mask'}
color_dict  = {0:(0,255,0),1:(0,0,255)}

while(True):
    
    ret,img = source.read()
    
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #cv2.imshow('frame', gray)

    faces = face_clsfr.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces:
        
        face_img = gray[ y:y+w, x:x+w ]
        
        resized = cv2.resize(face_img,(100,100))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,100,100,1))
        result = model.predict(reshaped)
        #print(result)
        
        label = np.argmax(result,axis=1)[0]
        print(label)
        
        cv2.rectangle(img,(x,y),(x+w,y+h), color_dict[label], 2)
        cv2.rectangle(img,(x,y-40),(x+w,y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)


    #cv2.imshow('LIVE',img)
    
    key=cv2.waitKey(1)

    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()